# Tests

## Imports & defaults

In [1]:
import cartopy.crs as ccrs
import libs.helpers as helpers
import libs.plot
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray

import warnings
warnings.filterwarnings('ignore')


def regional(data):
    regions = helpers.nsidc_regions()

    fig, axs = plt.subplots(
        3, 
        3, 
        figsize=(15, 15),
        subplot_kw={ 
            'projection': ccrs.Stereographic(central_latitude=90.0)
        }
    )
    transform = ccrs.PlateCarree()
    axs = axs.flatten()
    
    for i, region in enumerate(regions[1:]):
        data_masked = data.copy()\
            .sel(time=slice('2080-01-01', '2101-01-01'))\
            .where(np.isin(nsidc_mask.values, region['values']))\
            .mean(dim=('time'), skipna=True)
        
        ax = axs[i]
        ax.coastlines(resolution='110m', linewidth=0.5)
        ax.set_extent([-180, 180, 45, 90], transform)
        gl = ax.gridlines()
        gl.ylocator = matplotlib.ticker.LinearLocator(4)

        # Crop to circle
        theta = np.linspace(0, 2 * np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = matplotlib.path.Path(verts * radius + center)
        ax.set_boundary(circle, transform=ax.transAxes)
        data_masked.plot.pcolormesh(
            add_colorbar=False,
            ax=ax,
            levels=21,
            shading='flat',
            transform=transform
        )
        ax.set_title(region['label'])

## Regional masks

In [ ]:
path_nsidc_mask = '_data/_cache/NSIDC_Regions_Masks_LatLon_nearest_s2d.nc'
nsidc_mask = xarray.open_mfdataset(paths=path_nsidc_mask, combine='by_coords').mask
nsidc_mask = nsidc_mask.roll(x=96, roll_coords=True)

pr = helpers.get_data('ssp585', 'Amon', 'pr', 'r2i1p1f2').pr
pr = pr.where(pr.lat > 60).where(nsidc_mask.values > 0)

regional(pr)

## Weighted calculations

In [ ]:
path_areacella = '_data/cmip6/UKESM1/areacella_fx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
areacella = xarray.open_mfdataset(paths=path_areacella, combine='by_coords').areacella

pr_areacella_mean = pr[0, :, :].weighted(areacella).mean(dim=('lat', 'lon'))
pr_custom_mean = pr[0, :, :].weighted(np.cos(np.deg2rad(pr[0, :, :].lat))).mean(dim=('lat', 'lon'))
pr_areacella_multi_mean = (pr[0, :, :] * areacella).mean(dim=('lat', 'lon'))

print(pr_areacella_mean.values, pr_custom_mean.values, pr_areacella_multi_mean.values)

In [ ]:
pr_areacella_sum = pr[0, :, :].weighted(areacella).sum(dim=('lat', 'lon'))
pr_custom_sum = pr[0, :, :].weighted(np.cos(np.deg2rad(pr[0, :, :].lat))).sum(dim=('lat', 'lon'))
pr_areacella_multi_sum = (pr[0, :, :] * areacella).sum(dim=('lat', 'lon'))

print(pr_areacella_sum.values, pr_custom_sum.values, pr_areacella_multi_sum.values)